In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=splits, embedding=OllamaEmbeddings(model="mxbai-embed-large")
)

In [5]:
vectorstore._collection.get()

{'ids': ['388c7531-a90e-47da-8456-0703bde1c229',
  '0bff76ba-7c2e-44b5-bee6-5eb59cc04421',
  '34d3346c-60c6-4260-8696-7a7f61c8e735',
  '7beda998-dfe2-4cbb-a6cd-5549d4b1722f',
  '754375f8-355c-442c-8419-c0cb629aaedd',
  '1e1e2492-cd79-4054-9618-0b678e1c4447',
  'f4563a15-10ec-4146-9d27-fa77a2dff396',
  'aced34a9-c46e-462c-bc1b-4e1026a9dc5e',
  '663b2620-381b-484d-b7a9-28d5822c907c',
  '3dd2e94d-7fbc-4098-857d-18321872f4c6',
  'a38ee3cb-dc30-4d14-a2b5-bca79eb51fa3'],
 'embeddings': None,
 'documents': ['Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 16th September 2025Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · 3 classes/week · 2 hrs/class + Post-class Doubt SupportClasses on Tue, Wed, Thurs - 9PM ISTAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathe

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [8]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="gemma3:4b")

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [10]:
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

In [11]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'Yes, recordings of the course are available. You can access lifetime access to the recordings and also join the Discord community for support. The course is structured with 7 weeks of classes, and you’ll have post-class doubt support available.'